In [14]:
import pandas as pd
import numpy as np
import os
import xlwt
import re

In [15]:
GO = open("/Users/thesis/Desktop/Google Drive/thesis-lorenz/Work in progress.../Databases/GO/go-basic.obo", 'r')
GO.readline()

'format-version: 1.2\n'

In [17]:
for line in GO:
        print(line)

In [13]:
GO.close()

[Term]\n
